# GW 1 team selection

In [1]:
import os, sys, time
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()

pd.set_option('display.max_columns', None)

## 1. Previous season's data

In [2]:
data_folder = os.path.join('..', 'data', '2020-21')

### Basic player info

In [3]:
# read files and combine
players = pd.read_csv(os.path.join(data_folder, 'players.csv'))
positions = pd.read_csv(os.path.join(data_folder, 'positions.csv'))

players = players[
    ['player_id', 'first_name', 'second_name', 'web_name', 'position_id']
].merge(
    positions[['position_id', 'position_name']],
    on='position_id'
).drop(columns=['position_id'])

players.head()

,player_id,first_name,second_name,web_name,position_name
0,1,Mesut,Özil,Özil,MID
1,4,Pierre-Emerick,Aubameyang,Aubameyang,MID
2,9,Granit,Xhaka,Xhaka,MID
3,18,Nicolas,Pépé,Pépé,MID
4,19,Lucas,Torreira,Torreira,MID


### Player totals

In [4]:
# read gameweek history file
points = pd.read_csv(os.path.join(data_folder, 'gameweek_history.csv'))
# filter only gameweeks where player was involved
# add flag columns for minutes starter, bench and goals conceded
points = points[points['minutes']>0].assign(
    started=points['minutes'].apply(lambda x: 1 if x > 59 else 0),
    bench=points['minutes'].apply(lambda x: 1 if x < 60 else 0),
    conceded_2=points['goals_conceded'].apply(lambda x: x//2),
    saved_3=points['saves']//3
)

# aggregate player scores
points = points.groupby('player_id').sum().reset_index()
points.drop(columns=[
    'fixture', 'opponent_team', 'was_home', 'team_h_score', 'team_a_score',
    'round', 'value', 'transfers_balance', 'selected', 'transfers_in',
    'transfers_out'], inplace=True)
points.head(10)

,player_id,total_points,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,started,bench,conceded_2,saved_3
0,3,41,1396,1,0,3,20,0,0,0,1,1,0,0,230,250.6,46.7,144.0,43.9,13,7,6,0
1,4,131,2330,10,3,10,26,1,0,0,2,0,0,11,358,493.8,361.5,823.0,167.4,26,3,7,0
2,5,28,744,0,1,2,11,0,0,0,1,0,0,3,125,110.8,114.8,66.0,29.2,8,2,3,0
3,6,129,1916,13,3,7,21,0,0,0,3,0,0,21,452,602.4,307.4,797.0,170.4,19,12,4,0
4,7,3,45,0,0,0,1,0,0,0,0,0,0,0,16,11.6,1.0,0.0,1.3,0,3,0,0
5,8,131,3131,0,0,11,37,1,1,0,0,1,86,11,625,702.2,0.0,2.0,70.3,35,0,9,15
6,9,70,2519,1,2,10,27,0,0,0,7,1,0,1,396,456.0,327.6,154.0,94.0,27,4,6,0
7,10,34,900,0,0,4,8,0,0,0,2,0,0,1,182,163.2,26.4,16.0,20.6,10,0,1,0
8,11,94,2089,1,3,7,24,0,0,0,8,0,0,17,421,362.8,351.8,232.0,94.7,23,2,6,0
9,12,186,3420,0,0,15,46,0,1,0,1,0,142,27,825,1067.2,20.0,0.0,108.5,38,0,17,36


#### Join player info to aggregated scores

In [5]:
points = players.merge(points, on='player_id').sort_values('total_points', ascending=False)
points.drop(columns=['player_id', 'web_name'], inplace=True)
points.head(10)

,first_name,second_name,position_name,total_points,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,started,bench,conceded_2,saved_3
135,Bruno Miguel,Borges Fernandes,MID,244,3101,18,14,13,36,0,0,1,6,0,0,36,870,1292.6,1414.9,1253.0,396.2,34,3,10,0
469,Harry,Kane,FWD,242,3083,23,14,12,39,0,0,0,1,0,0,40,880,1318.2,659.1,1585.0,355.9,34,1,12,0
118,Mohamed,Salah,MID,231,3077,22,6,11,41,0,0,0,0,0,0,21,657,1056.0,825.7,1980.0,385.8,34,3,10,0
179,Heung-Min,Son,MID,228,3119,17,11,13,36,0,0,0,0,0,0,26,777,1052.2,1049.9,1046.0,315.2,34,3,10,0
442,Patrick,Bamford,FWD,194,3052,17,11,10,50,0,0,0,3,0,0,26,631,867.2,371.0,1512.0,274.6,32,6,16,0
439,Jamie,Vardy,FWD,187,2838,15,14,8,44,0,0,1,1,0,0,23,649,819.2,356.6,1306.0,248.3,31,3,15,0
486,Emiliano,Martínez,GKP,186,3420,0,0,15,46,0,1,0,1,0,142,27,825,1067.2,20.0,0.0,108.5,38,0,17,36
116,Sadio,Mané,MID,176,2808,11,11,12,28,0,0,0,3,0,0,14,513,808.8,809.7,1535.0,315.5,30,5,6,0
136,Marcus,Rashford,MID,174,2917,11,11,12,39,0,0,0,4,0,0,9,535,793.2,682.1,1127.0,260.3,32,5,12,0
303,Stuart,Dallas,DEF,171,3410,8,3,12,53,0,0,0,7,0,0,15,726,756.8,454.6,527.0,174.2,38,0,18,0


---
## 2. New season's data

In [6]:
data_folder = os.path.join('..', 'data', '2021-22')

### Basic player info

In [7]:
players = pd.read_csv(os.path.join(data_folder, 'players.csv'))
positions = pd.read_csv(os.path.join(data_folder, 'positions.csv'))
teams = pd.read_csv(os.path.join(data_folder, 'teams.csv'))

players = players[
    ['player_id', 'first_name', 'second_name', 'web_name', 'position_id',
     'now_cost', 'team_id']
].merge(
    positions[['position_id', 'position_name']],
    on='position_id'
).merge(
    teams[['team_id', 'team_name']],
    on='team_id'
)

players.head()

,player_id,first_name,second_name,web_name,position_id,now_cost,team_id,position_name,team_name
0,1,Bernd,Leno,Leno,1,50,1,GKP,Arsenal
1,2,Rúnar Alex,Rúnarsson,Rúnarsson,1,40,1,GKP,Arsenal
2,3,Willian,Borges Da Silva,Willian,3,65,1,MID,Arsenal
3,7,Granit,Xhaka,Xhaka,3,50,1,MID,Arsenal
4,12,Mohamed Naser,El Sayed Elneny,Elneny,3,45,1,MID,Arsenal


### Join previous season totals

In [8]:
df = players.merge(
    points, on=['first_name', 'second_name'], how='left', suffixes=('', '_old')
).fillna(0).set_index(['player_id'], drop=True)

df.head(10)

,first_name,second_name,web_name,position_id,now_cost,team_id,position_name,team_name,position_name_old,total_points,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,started,bench,conceded_2,saved_3
player_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,Bernd,Leno,Leno,1,50,1,GKP,Arsenal,GKP,131.0,3131.0,0.0,0.0,11.0,37.0,1.0,1.0,0.0,0.0,1.0,86.0,11.0,625.0,702.2,0.0,2.0,70.3,35.0,0.0,9.0,15.0
2,Rúnar Alex,Rúnarsson,Rúnarsson,1,40,1,GKP,Arsenal,GKP,1.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,8.0,16.6,0.0,0.0,1.7,0.0,1.0,0.0,0.0
3,Willian,Borges Da Silva,Willian,3,65,1,MID,Arsenal,MID,78.0,1398.0,1.0,7.0,3.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,306.0,256.2,534.8,228.0,102.3,15.0,10.0,4.0,0.0
7,Granit,Xhaka,Xhaka,3,50,1,MID,Arsenal,MID,70.0,2519.0,1.0,2.0,10.0,27.0,0.0,0.0,0.0,7.0,1.0,0.0,1.0,396.0,456.0,327.6,154.0,94.0,27.0,4.0,6.0,0.0
12,Mohamed Naser,El Sayed Elneny,Elneny,3,45,1,MID,Arsenal,MID,50.0,1538.0,1.0,0.0,5.0,20.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,258.0,200.2,99.9,79.0,37.8,17.0,6.0,4.0,0.0
13,Ainsley,Maitland-Niles,Maitland-Niles,3,55,1,MID,Arsenal,DEF,68.0,1714.0,0.0,2.0,6.0,23.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,273.0,218.2,128.9,142.0,48.8,19.0,7.0,6.0,0.0
15,Thomas,Partey,Partey,3,50,1,MID,Arsenal,MID,47.0,1529.0,0.0,2.0,6.0,18.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,273.0,289.4,195.7,129.0,61.7,16.0,8.0,4.0,0.0
17,Nicolas,Pépé,Pépé,3,75,1,MID,Arsenal,MID,114.0,1606.0,10.0,3.0,4.0,21.0,0.0,0.0,0.0,1.0,1.0,0.0,11.0,351.0,476.2,235.9,774.0,148.5,15.0,14.0,4.0,0.0
18,Lucas,Torreira,Torreira,3,45,1,MID,Arsenal,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Adjust player points to account for position changes

In [9]:
# new players will have 0 in 'position_name_old' column, replace with their current position
df[df['position_name_old']==0] = df[df['position_name_old']==0].assign(
    position_name_old=df['position_name']
)

In [10]:
df[df['position_name_old'] != df['position_name']]

,first_name,second_name,web_name,position_id,now_cost,team_id,position_name,team_name,position_name_old,total_points,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,started,bench,conceded_2,saved_3
player_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
13,Ainsley,Maitland-Niles,Maitland-Niles,3,55,1,MID,Arsenal,DEF,68.0,1714.0,0.0,2.0,6.0,23.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,273.0,218.2,128.9,142.0,48.8,19.0,7.0,6.0,0.0
4,Pierre-Emerick,Aubameyang,Aubameyang,4,100,1,FWD,Arsenal,MID,131.0,2330.0,10.0,3.0,10.0,26.0,1.0,0.0,0.0,2.0,0.0,0.0,11.0,358.0,493.8,361.5,823.0,167.4,26.0,3.0,7.0,0.0
150,Cheikhou,Kouyaté,Kouyaté,2,45,7,DEF,Crystal Palace,MID,82.0,3121.0,1.0,0.0,8.0,56.0,0.0,0.0,0.0,4.0,0.0,0.0,2.0,438.0,574.8,103.0,134.0,81.3,35.0,1.0,20.0,0.0
188,Stuart,Dallas,Dallas,3,55,10,MID,Leeds,DEF,171.0,3410.0,8.0,3.0,12.0,53.0,0.0,0.0,0.0,7.0,0.0,0.0,15.0,726.0,756.8,454.6,527.0,174.2,38.0,0.0,18.0,0.0
307,Allan,Saint-Maximin,Saint-Maximin,4,65,14,FWD,Newcastle,MID,85.0,1554.0,3.0,5.0,7.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,278.0,376.0,394.2,418.0,118.9,18.0,7.0,11.0,0.0
292,Matt,Ritchie,Ritchie,2,50,14,DEF,Newcastle,MID,52.0,1347.0,0.0,5.0,3.0,22.0,1.0,0.0,0.0,2.0,0.0,0.0,5.0,226.0,225.8,411.9,101.0,73.8,15.0,3.0,8.0,0.0


In [11]:
def calculate_points(player):
    points = (player['bench'] + 2*player['started'] + 4*player['goals_scored']
              + 3*player['assists'] + player['bonus'] - 3*player['red_cards']
              - player['yellow_cards'] - 2*player['penalties_missed']
              + 5*player['penalties_saved'] + player['saved_3'] - 2*player['own_goals'])
    
    if player['position_name'] == 'MID':
        # extra point per goal scored and one per clean sheet
        points += player['goals_scored'] + player['clean_sheets']
    
    elif player['position_name'] in ['DEF', 'GKP']:
        # extra 2 points per goal, 4 per clean sheet
        points += 2*player['goals_scored'] + 4*player['clean_sheets'] - 1*player['conceded_2']
    
    return points

In [12]:
df['adj_points'] = df.apply(calculate_points, axis=1)
df.head(10)

,first_name,second_name,web_name,position_id,now_cost,team_id,position_name,team_name,position_name_old,total_points,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,started,bench,conceded_2,saved_3,adj_points
player_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,Bernd,Leno,Leno,1,50,1,GKP,Arsenal,GKP,131.0,3131.0,0.0,0.0,11.0,37.0,1.0,1.0,0.0,0.0,1.0,86.0,11.0,625.0,702.2,0.0,2.0,70.3,35.0,0.0,9.0,15.0,131.0
2,Rúnar Alex,Rúnarsson,Rúnarsson,1,40,1,GKP,Arsenal,GKP,1.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,8.0,16.6,0.0,0.0,1.7,0.0,1.0,0.0,0.0,1.0
3,Willian,Borges Da Silva,Willian,3,65,1,MID,Arsenal,MID,78.0,1398.0,1.0,7.0,3.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,306.0,256.2,534.8,228.0,102.3,15.0,10.0,4.0,0.0,78.0
7,Granit,Xhaka,Xhaka,3,50,1,MID,Arsenal,MID,70.0,2519.0,1.0,2.0,10.0,27.0,0.0,0.0,0.0,7.0,1.0,0.0,1.0,396.0,456.0,327.6,154.0,94.0,27.0,4.0,6.0,0.0,70.0
12,Mohamed Naser,El Sayed Elneny,Elneny,3,45,1,MID,Arsenal,MID,50.0,1538.0,1.0,0.0,5.0,20.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,258.0,200.2,99.9,79.0,37.8,17.0,6.0,4.0,0.0,50.0
13,Ainsley,Maitland-Niles,Maitland-Niles,3,55,1,MID,Arsenal,DEF,68.0,1714.0,0.0,2.0,6.0,23.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,273.0,218.2,128.9,142.0,48.8,19.0,7.0,6.0,0.0,56.0
15,Thomas,Partey,Partey,3,50,1,MID,Arsenal,MID,47.0,1529.0,0.0,2.0,6.0,18.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,273.0,289.4,195.7,129.0,61.7,16.0,8.0,4.0,0.0,47.0
17,Nicolas,Pépé,Pépé,3,75,1,MID,Arsenal,MID,114.0,1606.0,10.0,3.0,4.0,21.0,0.0,0.0,0.0,1.0,1.0,0.0,11.0,351.0,476.2,235.9,774.0,148.5,15.0,14.0,4.0,0.0,114.0
18,Lucas,Torreira,Torreira,3,45,1,MID,Arsenal,MID,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
df[df['position_name_old'] != df['position_name']]

,first_name,second_name,web_name,position_id,now_cost,team_id,position_name,team_name,position_name_old,total_points,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,started,bench,conceded_2,saved_3,adj_points
player_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
13,Ainsley,Maitland-Niles,Maitland-Niles,3,55,1,MID,Arsenal,DEF,68.0,1714.0,0.0,2.0,6.0,23.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,273.0,218.2,128.9,142.0,48.8,19.0,7.0,6.0,0.0,56.0
4,Pierre-Emerick,Aubameyang,Aubameyang,4,100,1,FWD,Arsenal,MID,131.0,2330.0,10.0,3.0,10.0,26.0,1.0,0.0,0.0,2.0,0.0,0.0,11.0,358.0,493.8,361.5,823.0,167.4,26.0,3.0,7.0,0.0,111.0
150,Cheikhou,Kouyaté,Kouyaté,2,45,7,DEF,Crystal Palace,MID,82.0,3121.0,1.0,0.0,8.0,56.0,0.0,0.0,0.0,4.0,0.0,0.0,2.0,438.0,574.8,103.0,134.0,81.3,35.0,1.0,20.0,0.0,87.0
188,Stuart,Dallas,Dallas,3,55,10,MID,Leeds,DEF,171.0,3410.0,8.0,3.0,12.0,53.0,0.0,0.0,0.0,7.0,0.0,0.0,15.0,726.0,756.8,454.6,527.0,174.2,38.0,0.0,18.0,0.0,145.0
307,Allan,Saint-Maximin,Saint-Maximin,4,65,14,FWD,Newcastle,MID,85.0,1554.0,3.0,5.0,7.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,278.0,376.0,394.2,418.0,118.9,18.0,7.0,11.0,0.0,75.0
292,Matt,Ritchie,Ritchie,2,50,14,DEF,Newcastle,MID,52.0,1347.0,0.0,5.0,3.0,22.0,1.0,0.0,0.0,2.0,0.0,0.0,5.0,226.0,225.8,411.9,101.0,73.8,15.0,3.0,8.0,0.0,53.0


### Extrapolate points to every minute of season played

In [14]:
def extrapolate_points(player):
    points = player['adj_points']
    games = player['started'] + player['bench']
    if games != 0:
        proportion = games/38
        ppg = points/games
        ex_points = proportion*ppg
    else:
        ex_points = 0
    
    return ex_points

In [15]:
df = df.assign(
    points_per_game=round(df['adj_points'] / (df['started'] + df['bench']), 2),
    points_per_90=round(df['adj_points'] / df['minutes'] * 90, 2),
    now_cost=df['now_cost']/10
)

df['points_mixed'] = round((df['points_per_game'] + df['points_per_90']) / 2, 2)
df['ex_points_per_game'] = round(df.apply(extrapolate_points, axis=1), 2)

# adjust for players with low minutes
df[df['minutes']<450] = df[df['minutes']<450].assign(
    points_per_game=2.0,
    points_per_90=2.0,
    points_mixed=2.0
)

In [16]:
print('Top points per game')
df.sort_values('points_per_game', ascending=False).head(10)

Top points per game


,first_name,second_name,web_name,position_id,now_cost,team_id,position_name,team_name,position_name_old,total_points,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,started,bench,conceded_2,saved_3,adj_points,points_per_game,points_per_90,points_mixed,ex_points_per_game
player_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
357,Harry,Kane,Kane,4,12.5,17,FWD,Spurs,FWD,242.0,3083.0,23.0,14.0,12.0,39.0,0.0,0.0,0.0,1.0,0.0,0.0,40.0,880.0,1318.2,659.1,1585.0,355.9,34.0,1.0,12.0,0.0,242.0,6.91,7.06,6.98,6.37
276,Jesse,Lingard,Lingard,3,7.0,13,MID,Man Utd,MID,106.0,1423.0,9.0,5.0,4.0,19.0,0.0,0.0,1.0,3.0,0.0,0.0,15.0,332.0,487.0,288.2,530.0,130.4,16.0,0.0,5.0,0.0,106.0,6.62,6.70,6.66,2.79
277,Bruno Miguel,Borges Fernandes,Fernandes,3,12.0,13,MID,Man Utd,MID,244.0,3101.0,18.0,14.0,13.0,36.0,0.0,0.0,1.0,6.0,0.0,0.0,36.0,870.0,1292.6,1414.9,1253.0,396.2,34.0,3.0,10.0,0.0,244.0,6.59,7.08,6.84,6.42
233,Mohamed,Salah,Salah,3,12.5,11,MID,Liverpool,MID,231.0,3077.0,22.0,6.0,11.0,41.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,657.0,1056.0,825.7,1980.0,385.8,34.0,3.0,10.0,0.0,231.0,6.24,6.76,6.50,6.08
359,Heung-Min,Son,Son,3,10.0,17,MID,Spurs,MID,228.0,3119.0,17.0,11.0,13.0,36.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0,777.0,1052.2,1049.9,1046.0,315.2,34.0,3.0,10.0,0.0,228.0,6.16,6.58,6.37,6.00
252,John,Stones,Stones,2,5.5,12,DEF,Man City,DEF,128.0,1933.0,4.0,0.0,14.0,12.0,0.0,0.0,0.0,0.0,1.0,0.0,11.0,531.0,520.0,79.8,192.0,79.0,21.0,1.0,3.0,0.0,128.0,5.82,5.96,5.89,3.37
251,Kevin,De Bruyne,De Bruyne,3,12.0,12,MID,Man City,MID,141.0,1995.0,6.0,12.0,12.0,17.0,0.0,0.0,1.0,1.0,0.0,0.0,19.0,566.0,760.8,1170.1,914.0,284.7,22.0,3.0,5.0,0.0,141.0,5.64,6.36,6.00,3.71
250,Ilkay,Gündogan,Gündogan,3,7.5,12,MID,Man City,MID,157.0,2025.0,13.0,4.0,12.0,16.0,0.0,0.0,1.0,1.0,0.0,0.0,22.0,620.0,754.6,688.4,712.0,215.7,21.0,7.0,3.0,0.0,157.0,5.61,6.98,6.30,4.13
205,Jamie,Vardy,Vardy,4,10.5,9,FWD,Leicester,FWD,187.0,2838.0,15.0,14.0,8.0,44.0,0.0,0.0,1.0,1.0,0.0,0.0,23.0,649.0,819.2,356.6,1306.0,248.3,31.0,3.0,15.0,0.0,187.0,5.50,5.93,5.72,4.92


In [17]:
print('Top points per 90')
df.sort_values('points_per_90', ascending=False).head(10)

Top points per 90


,first_name,second_name,web_name,position_id,now_cost,team_id,position_name,team_name,position_name_old,total_points,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,started,bench,conceded_2,saved_3,adj_points,points_per_game,points_per_90,points_mixed,ex_points_per_game
player_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
265,Phil,Foden,Foden,3,8.0,12,MID,Man City,MID,135.0,1607.0,9.0,8.0,6.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,479.0,580.8,545.8,734.0,186.0,15.0,13.0,3.0,0.0,135.0,4.82,7.56,6.19,3.55
277,Bruno Miguel,Borges Fernandes,Fernandes,3,12.0,13,MID,Man Utd,MID,244.0,3101.0,18.0,14.0,13.0,36.0,0.0,0.0,1.0,6.0,0.0,0.0,36.0,870.0,1292.6,1414.9,1253.0,396.2,34.0,3.0,10.0,0.0,244.0,6.59,7.08,6.84,6.42
357,Harry,Kane,Kane,4,12.5,17,FWD,Spurs,FWD,242.0,3083.0,23.0,14.0,12.0,39.0,0.0,0.0,0.0,1.0,0.0,0.0,40.0,880.0,1318.2,659.1,1585.0,355.9,34.0,1.0,12.0,0.0,242.0,6.91,7.06,6.98,6.37
240,Diogo,Jota,Jota,3,7.5,11,MID,Liverpool,MID,86.0,1106.0,9.0,0.0,4.0,15.0,0.0,0.0,0.0,2.0,0.0,0.0,9.0,228.0,387.4,188.3,680.0,125.7,11.0,8.0,4.0,0.0,86.0,4.53,7.00,5.76,2.26
250,Ilkay,Gündogan,Gündogan,3,7.5,12,MID,Man City,MID,157.0,2025.0,13.0,4.0,12.0,16.0,0.0,0.0,1.0,1.0,0.0,0.0,22.0,620.0,754.6,688.4,712.0,215.7,21.0,7.0,3.0,0.0,157.0,5.61,6.98,6.30,4.13
213,Kelechi,Iheanacho,Iheanacho,4,7.5,9,FWD,Leicester,FWD,110.0,1452.0,12.0,2.0,5.0,23.0,0.0,0.0,1.0,1.0,0.0,0.0,18.0,424.0,540.2,413.9,814.0,176.5,16.0,9.0,6.0,0.0,110.0,4.40,6.82,5.61,2.89
233,Mohamed,Salah,Salah,3,12.5,11,MID,Liverpool,MID,231.0,3077.0,22.0,6.0,11.0,41.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,657.0,1056.0,825.7,1980.0,385.8,34.0,3.0,10.0,0.0,231.0,6.24,6.76,6.50,6.08
254,Riyad,Mahrez,Mahrez,3,9.0,12,MID,Man City,MID,145.0,1945.0,9.0,8.0,11.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,505.0,606.0,693.6,893.0,219.1,23.0,4.0,6.0,0.0,145.0,5.37,6.71,6.04,3.82
276,Jesse,Lingard,Lingard,3,7.0,13,MID,Man Utd,MID,106.0,1423.0,9.0,5.0,4.0,19.0,0.0,0.0,1.0,3.0,0.0,0.0,15.0,332.0,487.0,288.2,530.0,130.4,16.0,0.0,5.0,0.0,106.0,6.62,6.70,6.66,2.79


In [18]:
print('Top points per 90 / points per game average')
df.sort_values('points_mixed', ascending=False).head(10)

Top points per 90 / points per game average


,first_name,second_name,web_name,position_id,now_cost,team_id,position_name,team_name,position_name_old,total_points,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,started,bench,conceded_2,saved_3,adj_points,points_per_game,points_per_90,points_mixed,ex_points_per_game
player_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
357,Harry,Kane,Kane,4,12.5,17,FWD,Spurs,FWD,242.0,3083.0,23.0,14.0,12.0,39.0,0.0,0.0,0.0,1.0,0.0,0.0,40.0,880.0,1318.2,659.1,1585.0,355.9,34.0,1.0,12.0,0.0,242.0,6.91,7.06,6.98,6.37
277,Bruno Miguel,Borges Fernandes,Fernandes,3,12.0,13,MID,Man Utd,MID,244.0,3101.0,18.0,14.0,13.0,36.0,0.0,0.0,1.0,6.0,0.0,0.0,36.0,870.0,1292.6,1414.9,1253.0,396.2,34.0,3.0,10.0,0.0,244.0,6.59,7.08,6.84,6.42
276,Jesse,Lingard,Lingard,3,7.0,13,MID,Man Utd,MID,106.0,1423.0,9.0,5.0,4.0,19.0,0.0,0.0,1.0,3.0,0.0,0.0,15.0,332.0,487.0,288.2,530.0,130.4,16.0,0.0,5.0,0.0,106.0,6.62,6.70,6.66,2.79
233,Mohamed,Salah,Salah,3,12.5,11,MID,Liverpool,MID,231.0,3077.0,22.0,6.0,11.0,41.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,657.0,1056.0,825.7,1980.0,385.8,34.0,3.0,10.0,0.0,231.0,6.24,6.76,6.50,6.08
359,Heung-Min,Son,Son,3,10.0,17,MID,Spurs,MID,228.0,3119.0,17.0,11.0,13.0,36.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0,777.0,1052.2,1049.9,1046.0,315.2,34.0,3.0,10.0,0.0,228.0,6.16,6.58,6.37,6.00
250,Ilkay,Gündogan,Gündogan,3,7.5,12,MID,Man City,MID,157.0,2025.0,13.0,4.0,12.0,16.0,0.0,0.0,1.0,1.0,0.0,0.0,22.0,620.0,754.6,688.4,712.0,215.7,21.0,7.0,3.0,0.0,157.0,5.61,6.98,6.30,4.13
265,Phil,Foden,Foden,3,8.0,12,MID,Man City,MID,135.0,1607.0,9.0,8.0,6.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,479.0,580.8,545.8,734.0,186.0,15.0,13.0,3.0,0.0,135.0,4.82,7.56,6.19,3.55
254,Riyad,Mahrez,Mahrez,3,9.0,12,MID,Man City,MID,145.0,1945.0,9.0,8.0,11.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,505.0,606.0,693.6,893.0,219.1,23.0,4.0,6.0,0.0,145.0,5.37,6.71,6.04,3.82
251,Kevin,De Bruyne,De Bruyne,3,12.0,12,MID,Man City,MID,141.0,1995.0,6.0,12.0,12.0,17.0,0.0,0.0,1.0,1.0,0.0,0.0,19.0,566.0,760.8,1170.1,914.0,284.7,22.0,3.0,5.0,0.0,141.0,5.64,6.36,6.00,3.71


In [19]:
print('Top extrapolated points per game')
df.sort_values('ex_points_per_game', ascending=False).head(10)

Top extrapolated points per game


,first_name,second_name,web_name,position_id,now_cost,team_id,position_name,team_name,position_name_old,total_points,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,started,bench,conceded_2,saved_3,adj_points,points_per_game,points_per_90,points_mixed,ex_points_per_game
player_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
277,Bruno Miguel,Borges Fernandes,Fernandes,3,12.0,13,MID,Man Utd,MID,244.0,3101.0,18.0,14.0,13.0,36.0,0.0,0.0,1.0,6.0,0.0,0.0,36.0,870.0,1292.6,1414.9,1253.0,396.2,34.0,3.0,10.0,0.0,244.0,6.59,7.08,6.84,6.42
357,Harry,Kane,Kane,4,12.5,17,FWD,Spurs,FWD,242.0,3083.0,23.0,14.0,12.0,39.0,0.0,0.0,0.0,1.0,0.0,0.0,40.0,880.0,1318.2,659.1,1585.0,355.9,34.0,1.0,12.0,0.0,242.0,6.91,7.06,6.98,6.37
233,Mohamed,Salah,Salah,3,12.5,11,MID,Liverpool,MID,231.0,3077.0,22.0,6.0,11.0,41.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,657.0,1056.0,825.7,1980.0,385.8,34.0,3.0,10.0,0.0,231.0,6.24,6.76,6.50,6.08
359,Heung-Min,Son,Son,3,10.0,17,MID,Spurs,MID,228.0,3119.0,17.0,11.0,13.0,36.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0,777.0,1052.2,1049.9,1046.0,315.2,34.0,3.0,10.0,0.0,228.0,6.16,6.58,6.37,6.00
189,Patrick,Bamford,Bamford,4,8.0,10,FWD,Leeds,FWD,194.0,3052.0,17.0,11.0,10.0,50.0,0.0,0.0,0.0,3.0,0.0,0.0,26.0,631.0,867.2,371.0,1512.0,274.6,32.0,6.0,16.0,0.0,194.0,5.11,5.72,5.42,5.11
205,Jamie,Vardy,Vardy,4,10.5,9,FWD,Leicester,FWD,187.0,2838.0,15.0,14.0,8.0,44.0,0.0,0.0,1.0,1.0,0.0,0.0,23.0,649.0,819.2,356.6,1306.0,248.3,31.0,3.0,15.0,0.0,187.0,5.50,5.93,5.72,4.92
30,Emiliano,Martínez,Martínez,1,5.5,2,GKP,Aston Villa,GKP,186.0,3420.0,0.0,0.0,15.0,46.0,0.0,1.0,0.0,1.0,0.0,142.0,27.0,825.0,1067.2,20.0,0.0,108.5,38.0,0.0,17.0,36.0,186.0,4.89,4.89,4.89,4.89
230,Sadio,Mané,Mané,3,12.0,11,MID,Liverpool,MID,176.0,2808.0,11.0,11.0,12.0,28.0,0.0,0.0,0.0,3.0,0.0,0.0,14.0,513.0,808.8,809.7,1535.0,315.5,30.0,5.0,6.0,0.0,176.0,5.03,5.64,5.34,4.63
281,Marcus,Rashford,Rashford,3,9.5,13,MID,Man Utd,MID,174.0,2917.0,11.0,11.0,12.0,39.0,0.0,0.0,0.0,4.0,0.0,0.0,9.0,535.0,793.2,682.1,1127.0,260.3,32.0,5.0,12.0,0.0,174.0,4.70,5.37,5.04,4.58


## Save files as forecasts

In [20]:
data_folder = os.path.join('..', 'data', 'forecasts')

In [21]:
# points per game forecast
df[
    ['web_name', 'position_id', 'position_name', 'team_id', 'team_name',
     'now_cost', 'points_per_game']
].rename(
    columns={'points_per_game': '1_pts'}
).to_csv(
    os.path.join(data_folder, 'ppg.csv'))

In [22]:
# points per 90 forecast
df[
    ['web_name', 'position_id', 'position_name', 'team_id', 'team_name',
     'now_cost', 'points_per_90']
].rename(
    columns={'points_per_90': '1_pts'}
).to_csv(
    os.path.join(data_folder, 'pp90.csv'))

In [23]:
# points mixed forecast
df[
    ['web_name', 'position_id', 'position_name', 'team_id', 'team_name',
     'now_cost', 'points_mixed']
].rename(
    columns={'points_mixed': '1_pts'}
).to_csv(
    os.path.join(data_folder, 'pm.csv'))

In [24]:
# extrapolated points per game
df[
    ['web_name', 'position_id', 'position_name', 'team_id', 'team_name',
     'now_cost', 'ex_points_per_game']
].rename(
    columns={'ex_points_per_game': '1_pts'}
).to_csv(
    os.path.join(data_folder, 'xppg.csv'))

---
## Team selector

In [25]:
# workaround to add src sibling directory to current path
src_path = os.path.normpath(
    os.path.abspath(os.path.join(os.getcwd(), os.path.pardir, 'src')))
if (not (src_path in sys.path)) :
    sys.path.append(src_path)

import squad_select

In [26]:
model = squad_select.SelectionModel(forecasts_file=os.path.join(data_folder, 'xppg.csv'), gw=1)

In [27]:
model.forecasts

,web_name,position_id,position_name,team_id,team_name,now_cost,1_pts
player_id,,,,,,,
1,Leno,1,GKP,1,Arsenal,5.0,3.45
2,Rúnarsson,1,GKP,1,Arsenal,4.0,0.03
3,Willian,3,MID,1,Arsenal,6.5,2.05
7,Xhaka,3,MID,1,Arsenal,5.0,1.84
12,Elneny,3,MID,1,Arsenal,4.5,1.32
...,...,...,...,...,...,...,...
433,Jonny,2,DEF,20,Wolves,4.5,0.42
437,Semedo,2,DEF,20,Wolves,5.0,2.87
439,Kilman,2,DEF,20,Wolves,4.5,1.53


In [28]:
model.solve_optimal_squad()

NOTE: Initialized model gw1_100budget.
Total expected value for budget 100.0: 59.25


,Name,Pos_id,Pos,Team,Price,xP,lineup,captain,vicecaptain
0,Martínez,1,GKP,Aston Villa,5.5,4.89,1,0,0
2,Targett,2,DEF,Aston Villa,5.0,3.63,1,0,0
14,Cresswell,2,DEF,West Ham,5.5,4.03,1,0,0
7,Robertson,2,DEF,Liverpool,7.0,4.24,1,0,0
5,Dallas,3,MID,Leeds,5.5,3.82,1,0,0
11,Ward-Prowse,3,MID,Southampton,6.5,4.11,1,0,0
8,Harrison,3,MID,Man City,6.0,4.21,1,0,0
12,Son,3,MID,Spurs,10.0,6.00,1,0,0
9,Fernandes,3,MID,Man Utd,12.0,6.42,1,1,0
6,Bamford,4,FWD,Leeds,8.0,5.11,1,0,0
